<a href="https://colab.research.google.com/github/rogercs99/SisInteract_P5/blob/master/TexScaling_ESRGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# 0⃣  MONTA GOOGLE DRIVE
# ==========================================
from google.colab import drive
from subprocess import CalledProcessError, TimeoutExpired, run as sp_run
print("📀 Montando Google Drive…")
drive.mount('/content/drive')
print("✅ Drive montado.\n")

# ==========================================
# 1⃣  RUTAS BÁSICAS
# ==========================================
import os, json, time, shutil, datetime, subprocess, pathlib, uuid

BASE_DIR   = '/content/drive/MyDrive/TexScaling/Real-ESRGAN'
INPUT_ROOT = os.path.join(BASE_DIR, 'upload')
OUTPUT_ROOT= os.path.join(BASE_DIR, 'results')

print("📂 Rutas clave:")
print(f"   BASE_DIR   ➜ {BASE_DIR}")
print(f"   INPUT_ROOT ➜ {INPUT_ROOT}")
print(f"   OUTPUT_ROOT➜ {OUTPUT_ROOT}")

os.makedirs(INPUT_ROOT,  exist_ok=True)
os.makedirs(OUTPUT_ROOT, exist_ok=True)

%cd "{BASE_DIR}"

print("📦 Instalando dependencias (basicsr, facexlib, gfpgan)…")
!pip install -q basicsr facexlib gfpgan
print("📝 Instalando requirements.txt …")
!pip install -q -r requirements.txt
print("🔧 Instalando Real-ESRGAN en modo develop…")
!python setup.py -q develop
print("✅ Dependencias listas.\n")

# --- parche torchvision (Python 3.11) ---
patch_file = '/usr/local/lib/python3.11/dist-packages/basicsr/data/degradations.py'
old = 'from torchvision.transforms.functional_tensor import rgb_to_grayscale'
new = 'from torchvision.transforms.functional import rgb_to_grayscale'

if pathlib.Path(patch_file).exists():
    txt = pathlib.Path(patch_file).read_text()
    if old in txt:
        print("🩹 Parchando torchvision import en degradations.py…")
        pathlib.Path(patch_file).write_text(txt.replace(old, new))
        print("✅ Parche aplicado.\n")
    else:
        print("ℹ️  Parche ya aplicado previamente.\n")
else:
    print("⚠️  Archivo de parche no encontrado, continuamos.\n")

# ==========================================
# 3⃣  FUNCIÓN QUE PROCESA UNA CARPETA (con control robusto de errores)
# ==========================================
ESRGAN_TIMEOUT = 30 * 60          # ⏲️  30 min por carpeta (ajusta según tu GPU)
PRINT_CHUNK    = 200              # nº de primeros caracteres del stderr que se muestran

def run_esrgan_for(tx_id: str) -> bool:
    print(f"\n🔍 Revisando carpeta {tx_id}")
    in_dir       = os.path.join("upload", tx_id)        # relativa para ES RGAN
    out_dir      = os.path.join("results", tx_id)
    in_full      = os.path.join(INPUT_ROOT, tx_id)      # absoluta en Drive
    status_in    = os.path.join(in_full, 'status.json')
    log_fp       = os.path.join(out_dir, 'esrgan.log')
    output_full  = os.path.join(OUTPUT_ROOT, tx_id)
    status_out   = os.path.join(output_full, 'status.json')
    tmp_input    = f"/tmp/{tx_id}_imgs"                 # carpeta temporal solo imágenes
    tmp_output   = f"/tmp/{tx_id}_out"                  # carpeta temporal para ESRGAN

    print(f"   ↳ in_dir    : {in_dir}")
    print(f"   ↳ out_dir   : {out_dir}")
    print(f"   ↳ status_in : {status_in}")

    # 1) Leer status.json en input
    try:
        with open(status_in, 'r') as f:
            status = json.load(f)
        print(f"   ↳ Estado actual: {status.get('status')}")
    except Exception as e:
        print(f"   ❌ Error leyendo status.json → {e}")
        return False

    if status.get('status') != 'ready_to_process':
        print("   🚫 Estado ≠ ready_to_process, saltando.\n")
        return False

    # 2) Marcar "processing" en input
    status['status'] = 'processing'
    with open(status_in, 'w') as f:
        json.dump(status, f, indent=2)
    print("   ✍️  Marcado como processing en input.")

    # 3) Preparar carpeta temporal con solo imágenes
    print("   📂 Preparando carpeta temporal con solo imágenes")
    if os.path.exists(tmp_input):
        shutil.rmtree(tmp_input)
    os.makedirs(tmp_input, exist_ok=True)

    exts = (".png", ".jpg", ".jpeg")
    for fname in os.listdir(in_full):
        if fname.lower().endswith(exts):
            src = os.path.join(in_full, fname)
            dst = os.path.join(tmp_input, fname)
            shutil.copy2(src, dst)
            print(f"      ✔️ Copiado imagen: {fname}")
    print(f"   ➜ {len(os.listdir(tmp_input))} imágenes copiadas a {tmp_input}")

    # 4) Ejecutar Real-ESRGAN en tmp_input → tmp_output
    print("   🚀 Lanzando ESRGAN solo con imágenes")
    if os.path.exists(tmp_output):
        shutil.rmtree(tmp_output)
    os.makedirs(tmp_output, exist_ok=True)

    cmd = [
        'python',
        os.path.join(BASE_DIR, 'inference_realesrgan.py'),
        '-n', 'RealESRGAN_x4plus',
        '-i', tmp_input,
        '-o', tmp_output,
        '--outscale', '4',
        '--face_enhance'
    ]
    print(f"   ↳ Comando: {' '.join(cmd)} (timeout {ESRGAN_TIMEOUT}s)")

    try:
        result = subprocess.run(
            cmd,
            cwd=BASE_DIR,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            timeout=ESRGAN_TIMEOUT,
            check=False   # no interrumpir si exit != 0
        )
        # Guardar stdout+stderr en log
        os.makedirs(out_dir, exist_ok=True)
        with open(log_fp, 'w') as lf:
            lf.write(result.stdout)
            lf.write("\n--- STDERR ---\n")
            lf.write(result.stderr or "")

        if result.returncode == 0:
            print(f"   ✅ ESRGAN completó para {tx_id}")
        else:
            snippet = (result.stderr or "")[:PRINT_CHUNK]
            print(f"   ⚠️  ESRGAN exit {result.returncode}, continuando. "
                  f"Primeros {PRINT_CHUNK} chars stderr:\n{snippet}\n")
    except subprocess.TimeoutExpired as e:
        print(f"   ⏱️  Timeout ({ESRGAN_TIMEOUT}s) alcanzado")
        with open(log_fp, 'w') as lf:
            lf.write(e.stdout or "")
            lf.write("\n--- STDERR ---\n")
            lf.write(e.stderr or "")
    except FileNotFoundError as e:
        print(f"   ❌ inference_realesrgan.py no encontrado → {e}")

    # 5) Copiar resultados temporales a carpeta final SIN sufijo "_out"
    print("   📂 Moviendo resultados a output sin sufijo _out")
    os.makedirs(out_dir, exist_ok=True)
    renamed_count = 0
    for fname in os.listdir(tmp_output):
        # Solo imágenes: terminan en .png .jpg .jpeg (Real-ESRGAN crea *_out.png etc)
        if any(fname.lower().endswith(ext) for ext in exts):
            # Remover suffix "_out"
            new_name = fname.replace('_out', '')
            src = os.path.join(tmp_output, fname)
            dst = os.path.join(out_dir, new_name)
            shutil.move(src, dst)
            renamed_count += 1
            print(f"      ✔️ Renombrado {fname} → {new_name}")
    print(f"   ➜ {renamed_count} archivos movidos a {out_dir}")

    # 6) Contar imágenes en input y output
    print("   🔢 Contando imágenes en input y output…")
    input_count = len([f for f in os.listdir(tmp_input) if f.lower().endswith(exts)])
    output_count = len([f for f in os.listdir(out_dir)    if f.lower().endswith(exts)])
    print(f"   ↳ Imágenes en tmp_input/{tx_id}: {input_count}")
    print(f"   ↳ Imágenes en output/{tx_id}:   {output_count}")

    # 7) Verificar igualdad de conteos
    status_to_write = {
        "processed_at": datetime.datetime.utcnow().isoformat() + 'Z',
        "log_file":     log_fp
    }
    if input_count != output_count:
        print(f"   ⚠️ Conteos desiguales: input({input_count}) ≠ output({output_count})")
        status_to_write.update({
            "status":     "error",
            "error_code": "count_mismatch",
            "error_msg":  f"Conteo input {input_count} != output {output_count}"
        })
    else:
        status_to_write["status"] = "processed"
        print("   ✅ Conteos iguales.")

    # 8) Escribir status.json en carpeta output
    os.makedirs(output_full, exist_ok=True)
    with open(status_out, 'w') as f:
        json.dump(status_to_write, f, indent=2)
    print(f"   ✍️  status.json escrito en output: {status_out}\n")

    # 9) Limpiar temporales locales
    shutil.rmtree(tmp_input)
    shutil.rmtree(tmp_output)

    return True


# ==========================================
# 4⃣  BUCLE PRINCIPAL (polling)
# ==========================================
POLL_SECS = 1
loop_n    = 0
print("👀 Escuchando nuevas carpetas en upload/ … (Ctrl+C para parar)\n")

try:
    while True:
        loop_n += 1
        print(f"🔄 Iteración #{loop_n} — listando subcarpetas…")
        ids = [d for d in os.listdir(INPUT_ROOT)
               if os.path.isdir(os.path.join(INPUT_ROOT, d))]
        print(f"   Carpeta(s) detectada(s): {ids or 'ninguna'}")

        worked = False
        for tx in ids:
            worked |= run_esrgan_for(tx)

        if not worked:
            print("   ⏳ Nada para procesar, esperando…\n")

        time.sleep(POLL_SECS)

except KeyboardInterrupt:
    print("\n🛑 Bucle detenido manualmente.")


Se han truncado las últimas 5000 líneas del flujo de salida.
   ⏳ Nada para procesar, esperando…

🔄 Iteración #2392 — listando subcarpetas…
   Carpeta(s) detectada(s): ['d05756e5-bac2-4b99-b313-eacd4b067213']

🔍 Revisando carpeta d05756e5-bac2-4b99-b313-eacd4b067213
   ↳ in_dir    : upload/d05756e5-bac2-4b99-b313-eacd4b067213
   ↳ out_dir   : results/d05756e5-bac2-4b99-b313-eacd4b067213
   ↳ status_in : /content/drive/MyDrive/TexScaling/Real-ESRGAN/upload/d05756e5-bac2-4b99-b313-eacd4b067213/status.json
   ↳ Estado actual: processing
   🚫 Estado ≠ ready_to_process, saltando.

   ⏳ Nada para procesar, esperando…

🔄 Iteración #2393 — listando subcarpetas…
   Carpeta(s) detectada(s): ['d05756e5-bac2-4b99-b313-eacd4b067213']

🔍 Revisando carpeta d05756e5-bac2-4b99-b313-eacd4b067213
   ↳ in_dir    : upload/d05756e5-bac2-4b99-b313-eacd4b067213
   ↳ out_dir   : results/d05756e5-bac2-4b99-b313-eacd4b067213
   ↳ status_in : /content/drive/MyDrive/TexScaling/Real-ESRGAN/upload/d05756e5-bac2-4b99